In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import scipy.linalg
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# -----------------------------
# 0. Get user input or default
# -----------------------------
def get_input(prompt, default):
    try:
        val = input(prompt)
        return float(val) if val.strip() != '' else default
    except:
        return default

EC = get_input("Enter EC (default=1.0): ", 1.0)
EJ = get_input("Enter EJ (default=20.0): ", 20.0)
n_max = int(get_input("Enter n_max (default=30): ", 30))
time = get_input("Enter simulation time (default=1.0): ", 1.0)

print(f"\nUsing parameters: EC = {EC}, EJ = {EJ}, n_max = {n_max}, time = {time}")

# -----------------------------
# 1. Setup charge basis
# -----------------------------
n = np.arange(-n_max, n_max + 1)
N = len(n)

# -----------------------------
# 2. Build the Transmon Hamiltonian
# -----------------------------
H = np.zeros((N, N), dtype=complex)

# Diagonal (charging energy)
for i in range(N):
    H[i, i] = 4 * EC * n[i]**2

# Off-diagonal (Josephson tunneling)
for i in range(N - 1):
    H[i, i + 1] = H[i + 1, i] = -EJ / 2

# -----------------------------
# 3. Initial State |n=0⟩
# -----------------------------
psi0 = np.zeros(N, dtype=complex)
psi0[n_max] = 1.0  # |n=0⟩ at center

# -----------------------------
# 4. Time Evolution via Matrix Exponential
# -----------------------------
t_vals = np.linspace(0, 100, 2000)
dt = t_vals[1] - t_vals[0]
overlaps = []

for t in t_vals:
    U = scipy.linalg.expm(-1j * H * t)
    psi_t = U @ psi0
    overlap = np.vdot(psi0, psi_t)
    overlaps.append(np.abs(overlap)**2)

# -----------------------------
# 5. Fourier Transform (Spectrum)
# -----------------------------
spectrum = np.abs(np.fft.fft(overlaps))**2
freqs = np.fft.fftfreq(len(t_vals), d=dt)

# -----------------------------
# 6. Energy Levels from Diagonalization
# -----------------------------
energies = np.linalg.eigvalsh(H)
gaps = np.diff(energies[:5])

# Plot: Energy levels
plt.figure()
plt.plot(energies[:5], 'o-')
plt.title("Lowest 5 Energy Levels of a Transmon Qubit")
plt.xlabel("Level")
plt.ylabel("Energy")
plt.grid(True)
plt.tight_layout()
plt.show()

# Print gaps
print("Energy differences (from eigvalsh):")
for i, gap in enumerate(gaps):
    print(f"Level {i} → {i+1}: ΔE = {gap:.4f}")

# -----------------------------
# 7. Backend selection for 1-qubit demo
# -----------------------------
try:
    api_key = input('Enter your IBM API key: ')
    instance = input('Enter IBM instance name: ')
    backend_name = input('Enter IBM backend (e.g., ibm_brisbane): ')

    service = QiskitRuntimeService(channel="ibm_cloud", token=api_key, instance=instance)
    backend = service.backend(backend_name)
    print(f"✅ Using IBM backend: {backend_name}")
except Exception as e:
    print("⚠️ IBM backend not available, using AerSimulator.")
    backend = AerSimulator()

# -----------------------------
# 8. Simplified 1-Qubit Circuit Simulation
# -----------------------------
# Approximate Transmon Hamiltonian using 1-qubit circuit

theta = time * EJ / 2
phi = time * EC * 4

qc = QuantumCircuit(1, 1)
qc.h(0)
qc.rz(phi, 0)
qc.rx(theta, 0)
qc.measure(0, 0)


# Transpile and run on IBM backend
qc = transpile(qc, backend)
sampler = Sampler(backend)
job = sampler.run([qc], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()

# -----------------------------
# 9. Analyze and Plot
# -----------------------------
# Normalize counts to get probabilities
probs = {k: v / 1024 for k, v in counts.items()}

# Print
print("\nFinal state probabilities (1-qubit, from IBM backend):")
for state in ['0', '1']:
    print(f"|{state}>: {probs.get(state, 0.0):.4f}")

# Plot
plt.bar(probs.keys(), probs.values())
plt.ylabel("Probability")
plt.title("1-Qubit Transmon Time Evolution (IBM Backend)")
plt.grid(True)
plt.tight_layout()
plt.show()

# -----------------------------
# 10. Spectrum Plot from Full Simulation
# -----------------------------
plt.figure(figsize=(8, 4))
plt.plot(freqs[:len(freqs)//2], spectrum[:len(spectrum)//2])
plt.xlabel("Energy (frequency units)")
plt.ylabel("Spectral Intensity")
plt.title("Transmon Energy Spectrum via Time Evolution (Matrix Exp.)")
plt.grid(True)
plt.tight_layout()
plt.show()
